In [262]:
import glob
import pandas as pd
import numpy as np

file_name_list = glob.glob("Data/*.csv")

feature_set = ['wellName','DEPT', 'BS', 'CALI', 'DENS', 'DTC', 'GR', 'NEUT', 'PEF', 'RESD', 'RESM', 'RESS', 'TVD']

file_list = []

for file in file_name_list:
    df = pd.read_csv(file, index_col=None, skiprows=[1])
    file_list.append(df[feature_set])

# Algo 1

In [266]:
feature_set = ['DEPT', 'BS', 'CALI', 'DENS', 'GR', 'NEUT', 'PEF', 'RESD', 'RESM', 'RESS', 'TVD']

wells = []
MVE_list = []
    
for i in range(len(file_list)):
    test_df = file_list[i]
    wells.append(test_df.iloc[0,0])
    
    train_list = file_list.copy()
    train_list.pop(i)
    train_df = pd.concat(train_list)
    
    test_x = test_df[feature_set].values
    test_y = test_df[['DTC']].values
    
    train_X = train_df[feature_set].values
    train_y = train_df[['DTC']].values

    
      
    # feature scaling
    
    # training
    
    # testing
    MVE = .99
    
    
    
    MVE_list.append(MVE)

avg_MVE = np.mean(MVE_list)

for i in range(len(wells)):
    print('Test error on %s : %s' %(wells[i], MVE_list[i]))

print()
print('Average algorithm error: %s' %avg_MVE)


Test error on Cheal-A12 : 0.99
Test error on Cheal-G3 : 0.99
Test error on Cheal-B8 : 0.99
Test error on Cheal-G2 : 0.99
Test error on Cheal-A10 : 0.99
Test error on Cheal-C3 : 0.99
Test error on Cheal-G1 : 0.99
Test error on Cheal-A11 : 0.99
Test error on Cheal-C4 : 0.99

Average algorithm error: 0.99
